In [ ]:
import sys
from pathlib import Path

# Add project root to Python path
sys.path.insert(0, str(Path(__file__).parent.parent))

import pandas as pd
from store.analytics_store import AnalyticsStore
from builders.match_analytics_builder import MatchAnalyticsBuilder


# Load raw data
data_path = Path(__file__).parent.parent / "data" / "raw" / "season_df_v1.csv"
raw_df = pd.read_csv(data_path)

# Build MatchAnalytics objects for each unique match
builder = MatchAnalyticsBuilder()
match_analytics_list = []

for match_id in raw_df["match_id"].unique():
    match_df = raw_df[raw_df["match_id"] == match_id].copy()
    match_analytics = builder.build(match_df)
    match_analytics_list.append(match_analytics)

# Initialize store and ingest data
store = AnalyticsStore()
store.ingest(match_analytics_list)

print(f"Loaded {len(match_analytics_list)} matches into the store")

# Materialize all matches from the home perspective
all_home_matches = store.materialize(
    perspective="home",
    required_features=["home_normaltime"]  # This is the field for home goals
)

# Filter for home goals > 1
home_goals_gt_1 = all_home_matches[all_home_matches["home_normaltime"] > 1]

print(f"\nGames where home team scored > 1 goal:")
print(home_goals_gt_1)
print(f"\nTotal matches: {len(home_goals_gt_1)}")

ModuleNotFoundError: No module named 'store'